In [49]:
# -*- coding:utf-8 -*-
from elasticsearch import Elasticsearch, RequestError
from pprint import pprint as pp

es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

# Simple Filed types

1. String: string
2. Whole number: byte, short, integer, long
3. Floating-point: float, double
4. Boolean: boolean
5. Date: date

# Customizing Field Mappings

특히 string field에 대한 custom이 매우 중요합니다. Custom Mapping을 통해서 다음과 같은 일들을 할 수 있습니다.

1. Full-text string fields와 exact value string field를 구분합니다.
2. language-specific analyzers를 사용합니다.
3. partial matching에 optimize를 합니다.
4. custom date formats을 지정할수 있습니다.


1. **type**: 가장 중요한 요소는 **type**입니다. 기본적으로 string field는 full-text로 간주됩니다.<br> 즉 value는 indexed되기전에 analyzer를 거칠것이고, full-text query또한 타게 됩니다.
2. **index**: string이 어떻게 indexed 될 것인지 결정합니다. 그리고 3자기 요소를 갖고 있습니다.<br> - **analyzed**: string을 먼저 analyze하고 index를 합니다. 즉 full-text field가 됩니다. <br> - **not_analyzed**: 해당 필드를 index시킵니다. 즉 검색에 포함이됩니다. 하지만 정확하게 일치하는 검색이 됩니다. (analyze하지 않음) <br> - **no**: index하지 않습니다. 즉.. 검색되지 않습니다.
3. **analyzer**: string field이고 index는 analyzed일때, 검색및 index time때 적용할 analyzer를 명시할수 있습니다.<br> 종류로는 whitespace, simple, english가 기본적으로 제공됩니다.

```
 "tag": {
        "type":     "string",
        "index":    "not_analyzed",
        "analyzer": "english"
    }
```



# Create Index with mappings


### Viewing the mappings
아래의 코드를 실행후, [/test/_mapping](http://localhost:9200/test/_mapping)에 들어가보면 다음과 같은 결과를 얻을 수 있습니다.

```
{
    "test": {
        "mappings": {
            "types": {
                "properties": {
                    "about": {
                        "type": "string",
                        "analyzer": "simple"
                    },
                    "email": {
                        "type": "string",
                        "index": "not_analyzed"
                    },
                    "height": {
                        "type": "float"
                    }
                }
            }
        }
    }
}
```

### Mapping Structure
기본적으로 mappings을 할때 사용되는 JSON구조는 다음과 같습니다.

```
{
    'index 이름': {
        'mappings': {
            'type 이름': {
                'properties': {
                    ... fields 정의
                }
            }
        }

    }
}
```

In [18]:
if es.indices.exists(index='test'):
    es.indices.delete(index='test')
    
mapping = {
    'mappings': {
        'types': {
            'properties':{
                'email': {
                    'type': 'string',
                    'index': 'not_analyzed'
                },
                'height': {
                    'type': 'float'
                },
                'about': {
                    'type': 'string',
                    'index': 'analyzed',
                    'analyzer': 'simple'
                }    
            }
        }
    }
}
    
es.indices.create(index='test', ignore=400, body=mapping)

{u'acknowledged': True}

# 잘못된 type으로 index를 할때

위의 코드에서 mappings을 정의해주었습니다. 이때 잘못된 type을 index하려고 한다면 Elasticsearch는 Error를 내보냅니다.

In [27]:
try:
    es.create(index='test', doc_type='types', body={
            'email': 123,
            'height': 'Tall',
            'about': 'This is wrong'
        })
except RequestError as e:
    print e

TransportError(400, u'MapperParsingException[failed to parse [height]]; nested: NumberFormatException[For input string: "Tall"]; ')


In [28]:
es.create(index='test', doc_type='types', body={
        'email': 'a141890@gmail.com',
        'height': 183,
        'about': 'This is good'
    })

{u'_id': u'OojIU9lSSIiLZBSJNXKYbg',
 u'_index': u'test',
 u'_type': u'types',
 u'_version': 1,
 u'created': True}

# Updating a Mapping

기존의 Mapping에 새로운 Type으로 정의할수 있습니다.<br>
먼저 Elasticsearch가 자동으로 만들도록 합니다. <br>
[Mapping을 확인](http://localhost:9200/test/_mapping/up)해보면 다음과 같이 나옵니다.

```
{
    "test": {
        "mappings": {
            "up": {
                "properties": {
                    "age": {
                        "type": "long"
                    },
                    "height": {
                        "type": "double"
                    },
                    "hobby": {
                        "type": "string"
                    },
                    "name": {
                        "type": "string"
                    }
                }
            }
        }
    }
}

```

In [54]:
es.index(index='test', doc_type='up', id=1, body={
        'name': 'Anderson',
        'age': 33,
        'height': 182.2,
        'hobby': ['fishing', 'watching a movie', 'drive']
    })

{u'_id': u'1',
 u'_index': u'test',
 u'_type': u'up',
 u'_version': 1,
 u'created': True}

In [56]:
mappnig = {
    'up': {
        'properties': {
            'name': {'type': 'string', 'index': 'not_analyzed' },
            'age': {'type': 'integer'},
            'height': {'type': 'float'},
            'hobby': {'type': 'string', 'index': 'analyzed', 'analyzer': 'english'},
            'new': {'type': 'text'}
        }    
    }
}
es.indices.put_mapping(index='test', doc_type='up', body=mapping)

{u'acknowledged': True}